In [6]:
import os
import cv2
import mahotas
from collections import namedtuple
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import h5py

def hu_feature(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

def color(image, mask=None):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def texture(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

def extract_features(image_path):
    image = cv2.imread(image_path)
    Features = namedtuple('Features', 'hu color texture')
    features = Features(hu_feature(image),
                        color(image),
                        texture_feature(image),
                        )
    return np.hstack([features.hu, features.color, features.texture])

def normalization_features(features):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(features)

if __name__ == '__main__':
    train_dir = os.path.join(os.getcwd(), 'train')
    global_features = []
    labels = []
    for dir in os.listdir(train_dir):
        current_dir = os.path.join(train_dir, dir)
        for image_path in os.listdir(current_dir):
            if image_path.endswith(".jpg") or image_path.endswith(".png"):
                global_image_path = os.path.join(current_dir, image_path)
                img_features = extract_features(global_image_path)
                global_features.append(img_features)
                labels.append(dir)
        print(f'Изображения {dir} обработаны')

    rescaled_features = normalization_features(global_features)
    target = LabelEncoder().fit_transform(labels)
    with h5py.File('feature_values', 'w') as h5f_data:
        h5f_data.create_dataset('dataset_1', data=np.array(rescaled_features))
    with h5py.File('pokemon_names', 'w') as h5f_label:
        h5f_label.create_dataset('dataset_1', data=np.array(target))


Изображения Chansey обработаны
Изображения Grimer обработаны
Изображения Lickitung обработаны
Изображения Muk обработаны
Изображения Slowbro обработаны
Изображения Slowpoke обработаны
